## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import regex as re
from datetime import datetime
from dateutil.parser import parse

## Download

Retrieve the page

In [6]:
url = "https://www.amadorgov.org/services/covid-19/-fsiteid-1"

In [7]:
page = requests.get(url)

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [9]:
content = soup.find("div", id="widget_4_3778_3488")

Get timestamp

In [10]:
imgs = [
    c
    for c in content.find_all("img")
    if "Screenshot" and "Top" in c.attrs.get("alt", "")
]

In [11]:
url = f"https://www.amadorgov.org{imgs[0]['src']}"

IndexError: list index out of range

In [12]:
tz = pytz.timezone("America/Los_Angeles")

In [13]:
today = datetime.now(tz).date()

In [14]:
with open(data_dir / "amador" / f"{today}.png", "wb") as handler:
    handler.write(requests.get(url).content)